## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bukama,CD,-9.2000,25.8500,69.19,97,99,0.74,light rain
1,1,Hobart,AU,-42.8794,147.3294,85.26,54,20,1.99,few clouds
2,2,Port Alfred,ZA,-33.5906,26.8910,76.19,79,100,12.44,overcast clouds
3,3,Ushuaia,AR,-54.8000,-68.3000,42.46,75,40,10.36,scattered clouds
4,4,Avarua,CK,-21.2078,-159.7750,75.25,94,20,4.61,light rain


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hobart,AU,-42.8794,147.3294,85.26,54,20,1.99,few clouds
2,2,Port Alfred,ZA,-33.5906,26.8910,76.19,79,100,12.44,overcast clouds
4,4,Avarua,CK,-21.2078,-159.7750,75.25,94,20,4.61,light rain
5,5,Bengkulu,ID,-3.8004,102.2655,85.39,64,90,6.91,overcast clouds
16,16,Busselton,AU,-33.6500,115.3333,79.09,29,92,17.07,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
683,683,Victor Harbor,AU,-35.5667,138.6167,80.69,55,23,8.72,few clouds
687,687,Cam Ranh,VN,11.9214,109.1591,84.16,65,20,6.91,few clouds
690,690,Omboue,GA,-1.5746,9.2618,77.99,86,95,2.77,overcast clouds
693,693,Tual,ID,-5.6667,132.7500,83.68,78,100,22.17,heavy intensity rain


In [8]:
# 4a. Determine if there are any empty rows.
if preferred_cities_df.isnull().values.any() == True:
    print("Null values present in the Preferred cities list")

Null values present in the Preferred cities list


In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
if preferred_cities_df.isnull().values.any() == True:
    preferred_cities_df = preferred_cities_df.dropna()
clean_df = preferred_cities_df

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hobart,AU,85.26,few clouds,-42.8794,147.3294,
2,Port Alfred,ZA,76.19,overcast clouds,-33.5906,26.8910,
4,Avarua,CK,75.25,light rain,-21.2078,-159.7750,
5,Bengkulu,ID,85.39,overcast clouds,-3.8004,102.2655,
16,Busselton,AU,79.09,overcast clouds,-33.6500,115.3333,
17,Rikitea,PF,77.27,overcast clouds,-23.1203,-134.9692,
30,Meulaboh,ID,86.43,broken clouds,4.1363,96.1285,
31,Labuhan,ID,84.31,overcast clouds,-6.8844,112.2051,
32,Hermanus,ZA,78.91,overcast clouds,-34.4187,19.2345,
35,Atuona,PF,78.12,clear sky,-9.8000,-139.0333,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
# 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
# Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
# 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
# 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"Hotel not found... skipping.{index}")
  
        

Hotel not found... skipping.149
Hotel not found... skipping.503
Hotel not found... skipping.593
Hotel not found... skipping.648


In [16]:
# 7. Drop the rows where there is no Hotel Name.
hotel_clean_df = hotel_df.dropna()
hotel_clean_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hobart,AU,85.26,few clouds,-42.8794,147.3294,Mantra on Collins Hobart
2,Port Alfred,ZA,76.19,overcast clouds,-33.5906,26.8910,The Halyards Hotel
4,Avarua,CK,75.25,light rain,-21.2078,-159.7750,Paradise Inn
5,Bengkulu,ID,85.39,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
16,Busselton,AU,79.09,overcast clouds,-33.6500,115.3333,Observatory Guest House
...,...,...,...,...,...,...,...
683,Victor Harbor,AU,80.69,few clouds,-35.5667,138.6167,McCracken Country Club
687,Cam Ranh,VN,84.16,few clouds,11.9214,109.1591,OYO 598 Peony Hotel
690,Omboue,GA,77.99,overcast clouds,-1.5746,9.2618,Hotel Olako
693,Tual,ID,83.68,heavy intensity rain,-5.6667,132.7500,Grand Vilia Hotel


In [19]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_clean_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_clean_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))